In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

!ls /content/drive/MyDrive/Deepfake-GitHub


ModuleNotFoundError: No module named 'google.colab'

In [2]:
import os

# Path to your project folder
project_path = "/content/drive/MyDrive/Deepfake-GitHub"

# Walk through the folder and list all files
for root, dirs, files in os.walk(project_path):
    level = root.replace(project_path, '').count(os.sep)
    indent = ' ' * 4 * level
    print(f"{indent}{os.path.basename(root)}/")
    subindent = ' ' * 4 * (level + 1)
    for f in files:
        print(f"{subindent}{f}")


In [3]:
# Path to your existing requirements.txt
req_path = "/content/drive/MyDrive/Deepfake-GitHub/requirements.txt"

# Overwrite it with minimal requirements
req_content = """\
torch>=2.0.0
torchvision>=0.15.0
opencv-python
numpy
matplotlib
Pillow
google-colab
IPython
"""

with open(req_path, "w") as f:
    f.write(req_content)

print("requirements.txt updated!")


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Deepfake-GitHub/requirements.txt'

In [10]:
!pip install -r /content/drive/MyDrive/Deepfake-GitHub/requirements.txt


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 19.1 MB/s eta 0:00:00


In [4]:


# ======== 2. Import required libraries ========
import torch
import torchvision.models
from torchvision import transforms
from PIL import Image
import cv2
from google.colab.patches import cv2_imshow
from google.colab import files
import numpy as np

# ======== 3. Upload the video ========
print("Upload a video file:")
uploaded = files.upload()
video_path = list(uploaded.keys())[0]
print("Uploaded video:", video_path)

# ======== 4. Load trained model safely ========
# Allow MobileNetV2 class (used in your checkpoint)
torch.serialization.add_safe_globals([torchvision.models.mobilenetv2.MobileNetV2])

# Select your model file from Drive
model_files = [
    "Copy of deepfake_hybrid_model_full.pth",
    "Copy of deepfake_hybrid_epoch7.pth",
    "Copy of deepfake_hybrid_model.pth",
    "Copy of deepfake_model.pth"
]

print("\nAvailable model files:")
for i, f in enumerate(model_files, 1):
    print(f"{i}. {f}")

choice = int(input("Select a model to load (1-4): "))
model_path = f"/content/drive/MyDrive/Deepfake-GitHub/{model_files[choice-1]}"

# Load the model
model = torch.load(model_path, map_location='cpu', weights_only=False)
model.eval()
print(f"✅ Model '{model_files[choice-1]}' loaded successfully!")

# ======== 5. Define preprocessing ========
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# ======== 6. Open video and predict ========
cap = cv2.VideoCapture(video_path)
frame_count = 0
predictions = []
displayed_frames = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    if frame_count % 10 != 0:  # Skip some frames to speed up
        continue

    # Preprocess frame
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    pil_image = Image.fromarray(frame_rgb)
    input_tensor = preprocess(pil_image).unsqueeze(0)

    # Predict
    with torch.no_grad():
        output = model(input_tensor)
        pred = torch.argmax(output, dim=1).item()
        predictions.append(pred)

    # Display first 5 frames with smaller size
    if displayed_frames < 5:
        label = "FAKE" if pred == 1 else "REAL"
        color = (0, 0, 255) if pred == 1 else (0, 255, 0)

        height, width = frame.shape[:2]
        new_width = 480
        new_height = int(height * (new_width / width))
        frame_small = cv2.resize(frame, (new_width, new_height))

        frame_disp = cv2.putText(frame_small, f"{label}", (20, 50),
                                 cv2.FONT_HERSHEY_SIMPLEX, 1.2, color, 2)
        cv2_imshow(frame_disp)
        displayed_frames += 1

cap.release()

# ======== 7. Final video-level prediction ========
fake_count = sum(predictions)
total_count = len(predictions)
print(f"\nFrames analyzed: {total_count}")
print(f"Fake frames detected: {fake_count}")

if fake_count / total_count > 0.5:
    print("🚨 Video is likely FAKE")
else:
    print("✅ Video is likely REAL")


ModuleNotFoundError: No module named 'google.colab'

In [ ]:
# Mount Google Drive (if needed)
# from google.colab import drive
# drive.mount('/content/drive')

# Navigate to your repo folder
%cd /content/drive/MyDrive/Deepfake-GitHub


NameError: name 'drive' is not defined